<a href="https://colab.research.google.com/github/alyxxxv/alyxxxv/blob/main/Simulasi_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAVDESS Song Emotion Recognition

In [1]:
import glob
import os
import librosa
import numpy as np
import tensorflow as tf

In [2]:
!git clone https://github.com/bagustris/ravdess_song.git

Cloning into 'ravdess_song'...
remote: Enumerating objects: 1175, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1175 (delta 66), reused 39 (delta 15), pack-reused 1057
Receiving objects: 100% (1175/1175), 232.34 MiB | 24.69 MiB/s, done.
Resolving deltas: 100% (70/70), done.
Checking out files: 100% (1033/1033), done.


In [4]:
cd ravdess_song

[Errno 2] No such file or directory: 'ravdess_song'
/content/ravdess_song


In [5]:
# load extracted features
x_train = np.load('data/x_train.npy', allow_pickle=True)
x_test = np.load('data/x_test.npy', allow_pickle=True)
y_train = np.load('data/y_train.npy', allow_pickle=True)
y_test = np.load('data/y_test.npy', allow_pickle=True)


In [6]:
# reshape X for CNN
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)


In [7]:
def model_cnn():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization(axis=-1,
              input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(tf.keras.layers.Conv1D(256, 128, 1, padding='same'))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling1D(2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(6, activation='softmax'))

    # compile model: set loss, optimizer, metric
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

In [8]:
model = model_cnn()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 386, 1)            4         
_________________________________________________________________
conv1d (Conv1D)              (None, 386, 256)          33024     
_________________________________________________________________
activation (Activation)      (None, 386, 256)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 193, 256)          0         
_________________________________________________________________
flatten (Flatten)            (None, 49408)             0         
_________________________________________________________________
dropout (Dropout)            (None, 49408)             0         
_________________________________________________________________
dense (Dense)                (None, 6)                 2

In [9]:
# train CNN model
hist = model.fit(x_train, y_train, epochs=100, shuffle=True, batch_size=16)

Epoch 1/100
53/53 [==============================] - 4s 62ms/step - loss: 1.7870 - accuracy: 0.2608
Epoch 2/100
53/53 [==============================] - 3s 62ms/step - loss: 1.6628 - accuracy: 0.3074
Epoch 3/100
53/53 [==============================] - 3s 63ms/step - loss: 1.4063 - accuracy: 0.4127
Epoch 4/100
53/53 [==============================] - 3s 62ms/step - loss: 1.2596 - accuracy: 0.5072
Epoch 5/100
53/53 [==============================] - 3s 63ms/step - loss: 1.2442 - accuracy: 0.5156
Epoch 6/100
53/53 [==============================] - 3s 63ms/step - loss: 1.0481 - accuracy: 0.5789
Epoch 7/100
53/53 [==============================] - 3s 63ms/step - loss: 0.9439 - accuracy: 0.6411
Epoch 8/100
53/53 [==============================] - 3s 63ms/step - loss: 0.8555 - accuracy: 0.6770
Epoch 9/100
53/53 [==============================] - 3s 62ms/step - loss: 0.8238 - accuracy: 0.6627
Epoch 10/100
53/53 [==============================] - 3s 63ms/step - loss: 0.7948 - accuracy: 0.6878

In [10]:
evaluate = model.evaluate(x_test, y_test, batch_size=16)
print("Loss: ", evaluate[0], "--> Accuracy: ", evaluate[1])

11/11 [==============================] - 0s 14ms/step - loss: 0.8001 - accuracy: 0.7443
Loss:  0.8000785708427429 --> Accuracy:  0.7443181872367859
